

# Project: Investigate The Movie Database

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

I chose the movie dataset for my project. This dataset contain the information of over 10,000 movies! The questions that we are looking to answer are: Which genres are most popular from year to year? What kinds of properties are associated with movies that have high revenues?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

### General Properties
Some things to take note about our dataset. Any movie that the budget is 0 should be treated as missing values as per the documentation found [here](https://www.kaggle.com/tmdb/tmdb-movie-metadata). Also we are assuming that the values for budget, revenue, budget_adj, and revenue_adj are in USD. Finally the popularity is a metric used to help boost search results and discover options. [source](https://developers.themoviedb.org/3/discover/movie-discover) 

In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
df = pd.read_csv('tmdb-movies.csv')
df.head()

### Data Cleaning
1. Remove duplicated fields
2. Drop columns that aren't needed
3. Drop entries that are missing values

#### 1. Remove duplicated fields
Let's check to see if the dataset has any duplicate fields.

In [ ]:
print('Number of duplicated entries: {}'.format(df.duplicated().sum())) # the result is 1

Since the dataset has 1 duplicated field let's get rid of it.

In [ ]:
#dropping duplicated values
df.drop_duplicates(inplace=True)

#### 2. Drop columns that aren't needed
The next step is to drop columns that aren't needed to answer our questions.

__imbd_id:__ This is a unique value that helps id movies since we already have an id that is unique this field is not needed.

__budget and revenue:__ Both of these fields are being dropped because the dataset contains budget_adj and revenue_adj that has the values adjusted for inflation.

__homepage, taglines, overview:__ These columns contain data that aren't relevant to our questions.

In [ ]:
drop_list = ['imdb_id', 'budget', 'revenue', 'homepage', 'tagline', 'overview']
for c in drop_list:
    df.drop(columns = c, inplace=True)

#### 3. Remove missing values
Let's check if our dataframe has missing values

In [ ]:
#count the number of missing values
df.isna().sum()

Let's drop the rows with missing values.

In [ ]:
df.dropna(inplace=True)
df.info()

We also need to make sure that the movies budget_adj is not equal to 0

In [ ]:
df = df[df['budget_adj'] != 0]
df.info()

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Which genres are most popular from year to year?
To do this first we need to seperate the genres column to seperate columns.

In [ ]:
genres = df['genres'].str.split('|', expand=True)
genres.rename(columns = lambda x: 'genres'+ str(x), inplace=True)
genres_df = pd.concat([df,genres], axis=1)
genres_df.drop(labels='genres', inplace=True, axis=1)
genres_df.head()

Now we are going to use pandas crosstab function to count the number of times the genres appear and plot the results on a heatmap.

In [ ]:
q1 = pd.crosstab(genres_df['release_year'], genres_df['genres0'], dropna = False)
q2 = pd.crosstab(genres_df['release_year'], genres_df['genres1'], dropna = False)
q3 = pd.crosstab(genres_df['release_year'], genres_df['genres2'], dropna = False)
q4 = pd.crosstab(genres_df['release_year'], genres_df['genres3'], dropna = False)
q5 = pd.crosstab(genres_df['release_year'], genres_df['genres4'], dropna = False)
genres_count = q1.add(q2)
genres_count.add(q3)
genres_count.add(q4)
genres_count.add(q5)

plt.figure(figsize=(12,24))
sns.heatmap(genres_count,
            cmap = 'coolwarm', 
            robust = True, 
            annot = True, 
            cbar = False);

### What kinds of properties are associated with movies that have high revenues?
Let's filter our dataset by using only the last quartile filtered by the revenue_adj column

In [ ]:
genres_df['revenue_adj'].describe()

In [ ]:
high_rev = genres_df[genres_df['revenue_adj'] >= 1.267480e+08]
high_rev.info()

Now that we have a filtered version of our dataset let's check to see if all of these movies had a high budget.

In [ ]:
sns.distplot(high_rev['budget_adj'], axlabel='Budget in USD');

In [ ]:
sns.scatterplot(x='revenue_adj', y='budget_adj', data=high_rev);

Let's see if the movies with high revenue were profitable.

In [ ]:
high_rev['profit'] = df['revenue_adj'] - df['budget_adj']

In [ ]:
high_rev['profit'].describe()

In [ ]:
sns.distplot(high_rev['profit'],axlabel='Profit in USD');

In [ ]:
q1 = pd.crosstab(high_rev['release_year'], high_rev['genres0'], dropna = False)
q2 = pd.crosstab(high_rev['release_year'], high_rev['genres1'], dropna = False)
q3 = pd.crosstab(high_rev['release_year'], high_rev['genres2'], dropna = False)
q4 = pd.crosstab(high_rev['release_year'], high_rev['genres3'], dropna = False)
q5 = pd.crosstab(high_rev['release_year'], high_rev['genres4'], dropna = False)
genres_count = q1.add(q2)
genres_count.add(q3)
genres_count.add(q4)
genres_count.add(q5)

plt.figure(figsize=(12,24))
sns.heatmap(genres_count,
            cmap = 'coolwarm', 
            robust = True, 
            annot = True, 
            cbar = False);

<a id='conclusions'></a>
## Conclusions

Throughout the years we see that drama was the most popular genre of movies.

Some attributes of a movie with high revenue are:
They tended to have higher budgets, they were profitable and were mainly action, adventure, comedies or dramas.